In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
df = pd.read_csv('resultfilm.csv')

df_clean = df.copy()


In [24]:
# Trouver les titres qui apparaissent plus d'une fois
titres_dupliques = df_clean['titre'].value_counts()
titres_dupliques = titres_dupliques[titres_dupliques > 1].index.tolist()

print(f"Nombre de titres en double: {len(titres_dupliques)}")

# Parcourir chaque titre dupliqué et vérifier si le genre est cohérent
for titre in titres_dupliques:
    doublons = df_clean[df_clean['titre'] == titre]
    genres = doublons['genre_principale'].unique()
    
    print(f"\nTitre: {titre}")
    print(f"Nombre d'occurrences: {len(doublons)}")
    print(f"Genres: {genres}")
    
    # Vérifier si tous les genres sont identiques
    if len(genres) == 1:
        print("✓ Toutes les entrées ont le même genre")
    else:
        print("⚠️ Les entrées ont des genres différents")
    
    # Afficher les informations détaillées des doublons
    print(doublons[['titre', 'genre_principale', 'entrees_demarrage_france', 'date_sortie_france']])

Nombre de titres en double: 2065

Titre: L'Extraordinaire Voyage de Marona
Nombre d'occurrences: 3
Genres: ['Animation']
✓ Toutes les entrées ont le même genre
                                  titre genre_principale  \
0     L'Extraordinaire Voyage de Marona        Animation   
2064  L'Extraordinaire Voyage de Marona        Animation   
4130  L'Extraordinaire Voyage de Marona        Animation   

     entrees_demarrage_france date_sortie_france  
0                      12 065         08/01/2020  
2064                   12 065         08/01/2020  
4130                   12 065         08/01/2020  

Titre: La Vraie famille
Nombre d'occurrences: 3
Genres: ['Drame']
✓ Toutes les entrées ont le même genre
                 titre genre_principale entrees_demarrage_france  \
1068  La Vraie famille            Drame                   61 129   
3011  La Vraie famille            Drame                   61 129   
5194  La Vraie famille            Drame                   61 129   

     date_sortie

In [12]:
def clean_value(value):
    # Si la valeur est NaN ou une chaîne indiquant une valeur manquante
    if pd.isna(value) or value in ['Non disponible', '?', '? $', '- $', '-']:
        return np.nan
    
    # Si c'est une chaîne, on la nettoie et la convertit
    if isinstance(value, str):
        # Supprimer les symboles $, espaces, et remplacer les virgules par des points
        cleaned_value = value.replace('$', '').replace(' ', '').replace(',', '.')
        
        # Si après nettoyage, la chaîne est vide ou juste un tiret
        if cleaned_value == '' or cleaned_value == '-':
            return np.nan
        
        try:
            return float(cleaned_value)
        except ValueError:
            # Si la conversion échoue, on retourne NaN
            print(f"Impossible de convertir la valeur: '{value}' en nombre")
            return np.nan
    
    # Si ce n'est pas une chaîne, on retourne la valeur telle quelle
    return value

# Fonction pour extraire les minutes à partir d'une durée au format "1h 32min"
def extract_minutes(duree):
    if pd.isna(duree):
        return np.nan
    
    match = re.search(r'(\d+)h\s*(\d+)min', duree)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        return hours * 60 + minutes
    return np.nan

def clean_note_moyenne(value):
    if pd.isna(value) or value in ['Non disponible', 'note_moyenne']:
        return np.nan
    
    try:
        return float(value)
    except (ValueError, TypeError):
        print(f"Impossible de convertir la note moyenne: '{value}' en nombre")
        return np.nan
    

numeric_columns = ['entrees_demarrage_france', 'entrees_totales_france', 
                  'budget', 'recette_usa', 'recette_reste_du_monde', 'recette_monde']

for col in numeric_columns:
    df_clean[col] = df_clean[col].apply(clean_value)

df_clean['duree'] = df_clean['duree'].apply(extract_minutes)


df_clean['note_moyenne'] = df_clean['note_moyenne'].apply(clean_note_moyenne)

Impossible de convertir la valeur: 'entrees_demarrage_france' en nombre
Impossible de convertir la valeur: 'entrees_demarrage_france' en nombre
Impossible de convertir la valeur: 'entrees_totales_france' en nombre
Impossible de convertir la valeur: 'entrees_totales_france' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de convertir la valeur: '? ' en nombre
Impossible de co

In [13]:
#SUPPRIMER LES LIGNES OU LA TARGET EST NULL

df_clean = df_clean.dropna(subset=['entrees_demarrage_france'])

# Vérifier combien de lignes ont été conservées
print(f"Nombre de lignes après suppression des valeurs manquantes dans la target : {len(df_clean)}")

# Vérifier s'il reste des valeurs nulles dans la target
print(f"Nombre de valeurs nulles restantes dans la target : {df_clean['entrees_demarrage_france'].isna().sum()}")


Nombre de lignes après suppression des valeurs manquantes dans la target : 4594
Nombre de valeurs nulles restantes dans la target : 0


In [ ]:
#valeurs manquantes par colonnes
print(df_clean.isna().sum())

film_id                        0
titre                          0
genre_principale               0
date_sortie_france             0
date_sortie_usa                0
image_url                      0
synopsis                       6
duree                         19
note_moyenne                1435
acteurs                        0
entrees_demarrage_france       0
entrees_totales_france         0
budget                      3083
recette_usa                 2694
recette_reste_du_monde      1379
recette_monde               1261
dtype: int64


In [15]:
genre_counts = df_clean['genre_principale'].value_counts()

In [16]:
print("Nombre de films par genre :")
for genre, count in genre_counts.items():
    print(f"{genre}: {count} films")


Nombre de films par genre :
Drame: 1254 films
Animation: 690 films
Comédie: 597 films
Comédie dramatique: 341 films
Thriller: 308 films
Documentaire: 225 films
Horreur: 216 films
Aventure - Action: 183 films
Biopic: 144 films
Romance: 99 films
Science Fiction: 90 films
Film familial: 69 films
Adaptation Livre: 60 films
Fantasy: 57 films
Marvel: 42 films
Musical: 27 films
DC Comics: 21 films
Adapt. Jeu Vidéo: 18 films
Spectacle filmé: 15 films
Western: 15 films
Comicbook: 13 films
Vampires: 12 films
Adaptation TV: 12 films
Football: 11 films
Guerre: 9 films
Adaptation Théatre: 9 films
Disney réinventé: 9 films
Espionnage: 9 films
Film en 3D: 6 films
Adaptation BD: 6 films
2ème Guerre mondiale: 6 films
Péplum: 3 films
X - Erotique: 3 films
Univers Star Wars: 3 films
Univers X-Men: 3 films
Arts martiaux: 3 films
Basket Ball: 3 films
Catastrophe: 3 films


In [18]:
df_clean.drop_duplicates(inplace=True)

In [20]:
nombre_lignes = df_clean.shape[0]
print(nombre_lignes)

1670


In [21]:
# Créer une copie pour l'imputation
df_imputed = df_clean.copy()

# Définir un seuil minimum d'échantillons par genre
min_samples = 30

# Déterminer quels genres ont suffisamment d'échantillons
genre_counts = df_clean['genre_principale'].value_counts()
valid_genres = genre_counts[genre_counts >= min_samples].index.tolist()

# Imputation par genre pour les genres avec suffisamment d'échantillons
for genre in valid_genres:
    # Calculer les médianes pour ce genre
    genre_median_note = df_clean[df_clean['genre_principale'] == genre]['note_moyenne'].median()
    genre_median_budget = df_clean[df_clean['genre_principale'] == genre]['budget'].median()
    
    # Remplacer les valeurs manquantes pour ce genre
    mask = (df_clean['genre_principale'] == genre)
    df_imputed.loc[mask, 'note_moyenne'] = df_imputed.loc[mask, 'note_moyenne'].fillna(genre_median_note)
    df_imputed.loc[mask, 'budget'] = df_imputed.loc[mask, 'budget'].fillna(genre_median_budget)

# Pour les genres avec peu d'échantillons, utiliser la médiane globale
rare_genres_mask = ~df_clean['genre_principale'].isin(valid_genres)
df_imputed.loc[rare_genres_mask, 'note_moyenne'] = df_imputed.loc[rare_genres_mask, 'note_moyenne'].fillna(df_clean['note_moyenne'].median())
df_imputed.loc[rare_genres_mask, 'budget'] = df_imputed.loc[rare_genres_mask, 'budget'].fillna(df_clean['budget'].median())

# Vérifier qu'il ne reste pas de valeurs manquantes
print(f"Valeurs manquantes restantes dans note_moyenne: {df_imputed['note_moyenne'].isna().sum()}")
print(f"Valeurs manquantes restantes dans budget: {df_imputed['budget'].isna().sum()}")

Valeurs manquantes restantes dans note_moyenne: 0
Valeurs manquantes restantes dans budget: 0


In [22]:
df_imputed.to_csv('films_nettoyes.csv', index=False)